# Data Science for Mobility / Intro to Business Analytics

# Lecture 6 - Neural networks

<div>
    <br/>
    <img src="https://canlogger1000.csselectronics.com/img/intel/Predictive-Maintenance-4.0-Data-Logger-CANopen.png" width="400"/>
</div>

A major problem faced by businesses in asset-heavy industries such as manufacturing is the significant costs associated with delays in the production process due to mechanical problems. Most of these businesses are interested in predicting these problems in advance so that they can proactively fix these issues before they occur which will reduce the costly impact caused by downtime. 

In this notebook, we will try to predict machine failures in advance. Specifically, our goal is to **compute the probability that a machine will fail in the next 24 hours due to a certain component failure**. 

This notebook is based on dataset and data preparation provided in: https://gallery.azure.ai/Collection/Predictive-Maintenance-Implementation-Guide-1. If you are interested in understanding how the data was preprocessed, have a look at the notebook provided there. In today's lecture, we are focused more on the modelling part of the problem, particularly using neural networks!

Nowadays, there are many (really a lot!) of Python packages that allow to develop and train neural networks: e.g. Tensorflow, Torch, Keras (now actually part of Tensorflow), Theano, etc. Even Sklearn has support for neural networks. Unfortunately, the support in Sklearn is rather limiting. So, instead, we will rely on a very popular Python package that is still high-level and easy to use for the most part (like Sklearn), but much more powerful - Keras. Therefore, if you haven't done so, please install the Python package called "Keras" before you continue with this notebook.

In [ ]:
import numpy as np
import pandas as pd
import keras

%matplotlib inline
import matplotlib.pyplot as plt

#matplotlib style options
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)

# Loading data from the different sources

In order to build our predictive maintenance system, we will consider data from 5 sources:

* **Machine conditions and usage:** The operating conditions of a machine e.g. data collected from sensors.
* **Error history:** The history of error codes from the machine.
* **Maintenance history:** The repair history of a machine, e.g. previous maintenance activities or component replacements.
* **Machine features:** The features of a machine, e.g. engine size, make and model, location.
* **Failure history:** The failure history of a machine or component within the machine.

The raw data from these sources was already pre-processed, and relevant features were extracted from it in order to be used in this notebook. Therefore, all you need to do is to load the 5 datasets and make sure that you understand what features each one contains.

### Machine conditions and usage: telemetry data (voltage, rotation, pressure and vibration)

We begin by loading the CSV file with the telemetry data. It corresponds to sensor data and includes voltage, rotation, pressure and vibration. The raw data was very high-resolution (every 1h) for the problem that we are trying to solve (predict failures 24h in advance). Therefore, for each sensor, features were extracted corresponding to the mean ("mean") and standard deviation ("sd") of the sensor in last 3h, and also in the last 24h window. 

Can you load the "telemetry.csv" file using Pandas, convert the "datetime" column to the Python datetime basic format using pd.to_datetime(), and print the first rows of the dataset?

Make sure that you understand what the Pandas dataframe above contains. The column names should be more or less self-explainatory, but if it is not clear, make sure to ask! :-)

Can you make a plot for visualizing the voltage mean ("voltmean_3h") for the machine with ID = 1?

There are some higher spikes in the voltage every once in a while, right? Maybe they correlate with failures...

### Error data

The second major data source is the error logs. These are non-breaking errors thrown while the machine is still operational and do not constitute as failures. Each machine has different components, each of which can fail at any moment. However, before they fail, it is common for a machine to start registering error codes indicating that something may not be well. The file "error_count.csv" includes error counts since the last failure.

Let's load the data and see what it looks like:

In [ ]:
# load data from csv
error_count = pd.read_csv('error_count.csv')

# format datetime field which comes in as string
error_count['datetime'] = pd.to_datetime(error_count['datetime'], format="%Y-%m-%d %H:%M:%S")

# show first rows
error_count.head()

In [ ]:
# visualize error count data for one machine
error_count[error_count.machineID == 1].plot(x='datetime', 
                                             y=['error1count','error2count','error3count','error4count','error5count'],
                                             figsize=(20,8))
plt.ylabel('error count')
plt.show()

### Maintenance data

These are the scheduled and unscheduled maintenance records which correspond to both regular inspection of components as well as failures. A record is generated if a component is replaced during the scheduled inspection or replaced due to a breakdown. 

The file "comp_rep.csv" contains features extracted from the raw maintenance log data indicating the elapsed time since each component was replaced. Each machine has 4 components, hence the 4 "comp" columns in this dataset. Let's load it with Pandas and make a plot of the days since last component replacement over time for machineID = 1.

In [ ]:
# load data from csv
comp_rep = pd.read_csv('comp_rep.csv')

# format datetime field which comes in as string
comp_rep['datetime'] = pd.to_datetime(comp_rep['datetime'], format="%Y-%m-%d %H:%M:%S")

# show first rows
comp_rep.head()

In [ ]:
# visualize days since last component replacement
comp_rep[comp_rep.machineID == 1].plot(x='datetime', 
                                             y=['comp1','comp2','comp3','comp4'],
                                             figsize=(20,8))
plt.ylabel('days since last component replacement')
plt.show()

### Machine characteristics

The next dataset ("machines.csv") is rather simple: it contains, for each machine, information about the model and how old it is. Let's see what it looks like:

In [ ]:
# load data from csv
machines = pd.read_csv('machines.csv')

# show first rows
machines.head()

In [ ]:
# turn "model" variable into dummy variables
machines['model'] = machines['model'].astype('category')
machines = pd.get_dummies(machines)
machines.head()

### Failure data

Lastly, the most important data source: the failure logs. This is registry of dates and times when each of the components (1-4) failed. 

In [ ]:
# load data from csv
failures = pd.read_csv('failures.csv')

# format datetime field which comes in as string
failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")
#failures['failure'] = failures['failure'].astype('category')

print("Total number of failures: %d" % len(failures.index))
failures.head()

Can you make a plot of the distribution of the failures by component? Is it an even distribution? Or is one component much more prone to failures than the others?

# Combine different sources of data

Did you notice that dataset from the different sources above have different time scales? Telemetry and error data is provided every 3h, maintenance data is hourly, machine information is time-invariant, and failure data is just a log with the occurrences. In order to combine all these different sources of data and use them to forecast component failures, we need to make sure that we align them properly in the temporal dimension!

In order to do this merging and alignment, we can make use of the "merge" function of Pandas. For the purpose of this notebook, we will adopt a time resolution of 3h. Therefore, we need to be careful with the order in which we perform the "merge" operations, and also the keys that we use to merge ("on" parameter) and the type of join ("how" parameter)! The code below makes the data fusion and alignment at a 3h resolution. Please ensure that it makes sense for you.

Given this data preparation, the prediction problem then corresponds to making predictions every 3h about whether each machine component will fail in the next 24h.

In [ ]:
features = telemetry.merge(error_count, on=['datetime', 'machineID'], how='left')
features = features.merge(comp_rep, on=['datetime', 'machineID'], how='left')
features = features.merge(machines, on=['machineID'], how='left')

print(features.head())
features.describe()

### Prepare target variables

Now it is time to prepare the target variables, i.e. for each record, is there a failure in the next 24h? This is done by taking a time window prior to the failure of an asset and labelling the feature records that fall into that window as "about to fail due to a problem" while labelling all other records as "no failure". Note that this time window should be picked according to the business case: in some situations it may be enough to predict failures hours in advance, while in others days or weeks may be needed to allow e.g. for arrival of replacement parts.

In [ ]:
# main target variable: "failure"
labeled_features = features.merge(failures, on=['datetime', 'machineID'], how='left')
labeled_features = labeled_features.fillna(method='bfill', limit=7) # fill backward up to 24h
labeled_features = labeled_features.fillna('none')
labeled_features.head()

The prediction problem for this example scenerio is to estimate the probability that a machine will fail in the near future due to a failure of a certain component. More specifically, the goal is to compute the probability that a machine will fail in the next 24h due to a certain component failure (component 1, 2, 3, or 4). 

We could approach this problem as multi-class problem, where for each machine at every time-step we predict one of the following classes: "no failure", "failure in comp1", "failure in comp2", "failure in comp3", "failure in comp4". However, doing so entails the assumption that two components cannot fail simultaneously. For example, although related, "failure in comp1" and "failure in comp2" should be treated an independent events, rather then competing events - as you be the case if we treated this as a multi-class problem. Instead, we treat it as multiple binary classification problems. Therefore, we proceed by contructing multiple binary target variables (one per componenent), indicating whether each component will fail in the next 24h:

In [ ]:
# convert "failure" target variables into multiple binary targets 
# i.e. one per component indicating failure/no failure
labeled_features['comp1_fail'] = (labeled_features['failure'] == 'comp1').astype(int)
labeled_features['comp2_fail'] = (labeled_features['failure'] == 'comp2').astype(int)
labeled_features['comp3_fail'] = (labeled_features['failure'] == 'comp3').astype(int)
labeled_features['comp4_fail'] = (labeled_features['failure'] == 'comp4').astype(int)
labeled_features.head()

# Train/validation/test split

Let us now prepare the features $\textbf{x}$ and target variables y to be used for training, validation and testing. Note the reference to the **validation set**. I.e. besides training and testing, we further split the training set by leaving aside some its data for validation, as shown in the following figure:

<div>
    <br/>
    <img src="http://mlsm.man.dtu.dk/wp-content/uploads/2019/10/val_set.png" width="300"/>
</div>

Why is this important? When using neural networks, besides running the training algorithm, we have quite a few "design" choices to make. This includes things like neural network architecture (e.g. number of hidden layer, number of neurons per layer), activation functions, assessing convergence (e.g. early stopping of training to avoid overfitting), hyper-parameters, optimization algorithm, etc. We cannot simply choose the combination that yields the best results on the testset, because in doing so we risk choosing a combination that happens to work well on that particular testset (for some particular reason or by chance...), but might not be the best combination in general. In other words, we risk biasing our results, thus making them over-optimistic. We therefore need to distance ourselves from the testset, and only use it for the final evaluation of the performance. All other choices need to be made based on the validation set.

To make things easier, we will start by just considering one of the components: component 1. The target variables will then be whether or not component 1 will fail in the next 24h. For this purpose, we will use the following set of features:

In [ ]:
features_to_use = ['voltmean_3h', 'rotatemean_3h', 'pressuremean_3h', 'vibrationmean_3h', # telemetry features (3h)
                   'voltsd_3h', 'rotatesd_3h', 'pressuresd_3h', 'vibrationsd_3h', 
                   'voltmean_24h', 'rotatemean_24h', 'pressuremean_24h', 'vibrationmean_24h', # telemetry feat (24h)
                   'voltsd_24h', 'rotatesd_24h', 'pressuresd_24h', 'vibrationsd_24h', 
                   'error1count', 'error2count', 'error3count', 'error4count', 'error5count', # errors over last 24h
                   'comp1', 'comp2', 'comp3', 'comp4', # days since last component replacement
                   'model_model1', 'model_model2', 'model_model3', 'model_model4', 'age'] # machine characteristics

target_variables = 'comp1_fail' # whether or not component 1 will fail in the next 24h

It is now time to split our dataset in training, validation and test sets. We will use for training all the data until 2015-05-31 01:00:00. Data between 2015-06-01 01:00:00 and 2015-07-31 01:00:00 will be used for validation. And data after 2015-08-01 01:00:00 for testing. 

Can you prepare the data according to this split?

In [ ]:
# train/val/test split
X_train = # TODO
y_train = # TODO
X_val = # TODO
y_val = # TODO
X_test = # TODO
y_test = # TODO

In [ ]:
print("Num train examples:", len(y_train))
print("Num validation examples:", len(y_val))
print("Num test examples:", len(y_test))

Lets not forget to **standardize our data**! In general, standardizing your data for machine learning algorithms is always recommended!

In [ ]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# Baseline model: Logistic regression

It is always a good idea to start by setting up a baseline. Since we are considering neural networks, and obvious choice for baseline is a logistic regression model. Note that logistic regression corresponds to a neural network with no hidden layers (just input and output layers).

**An important note about model evaluation:**

But before we do that, let us create a function that evaluates the performance of the different classifiers by computing various classification error metrics. In predictive maintenance, machine failures are usually rare occurrences in the lifetime of the assets compared to normal operation. This causes an imbalance in the label distribution which usually causes poor performance as algorithms tend to classify majority class examples better at the expense of minority class examples as the total misclassification error is much improved when majority class is labeled correctly. This makes accuracy a terrible indicator of performance in this particular problem. Since the dataset is so unbalanced (99% of the observations correponds to non-failures), it is easy to get 99% accuracy simply by predicting "no failure" all the time. Therefore, we must rely on other (more robust) metrics such as precision, recall and F1-score (https://en.wikipedia.org/wiki/F1_score).

Make sure that you understand the following code:

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

# function to evaluate predictions
def evaluate(y_true, y_pred):
    # calculate and display confusion matrix
    labels = np.unique(y_true)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print('Confusion matrix\n- x-axis is true labels (none, comp1, etc.)\n- y-axis is predicted labels')
    print(cm)

    # calculate precision, recall, and F1 score
    accuracy = float(np.trace(cm)) / np.sum(cm)
    precision = precision_score(y_true, y_pred, average=None, labels=labels)[1]
    recall = recall_score(y_true, y_pred, average=None, labels=labels)[1]
    f1 = 2 * precision * recall / (precision + recall)
    print("accuracy:", accuracy)
    print("precision:", precision)
    print("recall:", recall)
    print("f1 score:", f1)

Now it is time to fit the logistic regression classifier. Can you do it using Sklearn?

Can you make predictions for the test set and evaluate their quality using the "evaluate" function defined above?

What do you think of the results? Not too bad, right? 

# Neural networks in Keras

<div>
    <br/>
    <img src="http://mlsm.man.dtu.dk/wp-content/uploads/2019/10/nn_2layer-1.png" width="400"/>
</div>

It is now time to try out a neural network classifier using Keras. Bulding a multi-layer neural network in Keras is fairly easy. We start by creating an object of the class "Sequential" (indicating that the neural network consists of sequence of layers):

```python
model = Sequential()
```

Now we can add layers to our neural network model. For this entire lecture, we will only consider fully connected (dense) layers. Other types exist, but are out of the scope.

```python
model.add(Dense(50, input_dim=30, activation='relu'))
```

The code above adds a new layer to the model with 50 neurons with a ReLU activations function. Since this is the first layer, we also need to tell Keras the dimension of the input (in this case our x's are 30-dimensional - we have 30 features in our dataset). We can now add more hidden layers, or add the final Dense layer. Note that since this is a binary classification problem, the last layer (output layer) must necessarily have only 1 neuron with a sigmoid activation:

```python
model.add(Dense(1, activation='sigmoid'))
```

We can also add Dropout (with a given rate) between the layers using:

```python
model.add(Dropout(rate=0.5))
```

Once the neural network is fully specified, we need to compile it and tell Keras what loss function to use (in the case of binary classification, the loss should be "binary_crossentropy" - see lecture slides), which optimizer to use (Keras has many, but "adam" is a very popular one; it uses an adaptive step size and generally performs very well), and what metrics that can be used to evaluate performance during training. 

```python
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
```

Once compiled, we can train the neural network model using the function "fit":

```python
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_val,y_val))
```

Note that we also specified the number of epochs (i.e. how many passes through the dataset), the batch_size (i.e. size of the batch of observations that is used to compute gradients in each gradient step during the optimization of the loss function), and the validation data (so that we can know how validation accuracy evolves over time during training).

Once trained, we can make predictions for new data using the method "predict":

```python
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int) # convert output of sigmoid to binary variable (0 or 1)
```

Note that the second line is necessary in order to convert the output of the sigmoid function in the last layer of the neural network to a binary variable (0 or 1).

And that's all! Not that very complicated, is it? Can you try to do it yourself for the predictive maintenance dataset? See if you can come up with a good neural network architecture that outperforms the logistic regression classifier, and try to gain some insights during the process about what works and what doesn't...

Note: Don't forget to make predictions on the validation set and evaluate them using for example the F1-score, to see how you are doing, and use that indicator to guide your choice of neural networks architecture and other hyper-parameters of the network. Never rely on testset performance for this! That is cheating! :-)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
# TODO

How are the results? Are you satistied with the neural network that you have developed? If yes, then it is time to test it on the testset and see how it performs compared to the logistic regression classifier:

In [ ]:
# evaluate the keras model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# evaluate predictions
print("- Test set results:")
evaluate(y_test, y_pred)

Did you managed to outperform the logistic regression classifier? By how much? How much money do you think one can save in costs associated with delays in the production process due to mechanical problems thanks to this improvement? This is certainly an interesting question worth investigating, don't you think? ;-)